In [1]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('Scenario-A-merged_5s.csv')
df2 = pd.read_csv('Scenario-B-merged_5s.csv')
label = 'label'
# Concatenate the DataFrames 
full_df = pd.concat([df1, df2], ignore_index=True)

#change labels tor=1 other=0
full_df.loc[full_df[label] == 'TOR', 'label'] = 1
full_df.loc[full_df[label] != 1, 'label'] = 0
full_df.head(5)

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,53913,216.58.208.46,80,6,435,0.0,4597.701149,435.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,10.0.2.15,53913,216.58.208.46,80,6,259,0.0,7722.007722,259.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,10.0.2.15,53913,216.58.208.46,80,6,891,0.0,2244.668911,891.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,10.0.2.15,53913,216.58.208.46,80,6,1074,0.0,1862.197393,1074.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,10.0.2.15,53913,216.58.208.46,80,6,315,0.0,6349.206349,315.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
#Reformatt IP's by removing all decimals accept for the 1st one
def format_ip(ip_address):
    parts = ip_address.split('.')
    formatted_ip = '.'.join([parts[0], ''.join(parts[1:])])
    return float(formatted_ip)

full_df['Source IP'] = full_df['Source IP'].apply(format_ip)
full_df[' Destination IP'] = full_df[' Destination IP'].apply(format_ip)
full_df.head(5)

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0215,53913,216.582085,80,6,435,0.0,4597.701149,435.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,10.0215,53913,216.582085,80,6,259,0.0,7722.007722,259.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,10.0215,53913,216.582085,80,6,891,0.0,2244.668911,891.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,10.0215,53913,216.582085,80,6,1074,0.0,1862.197393,1074.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,10.0215,53913,216.582085,80,6,315,0.0,6349.206349,315.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Calculate standard deviation for each column
std_dev = full_df.std()

# Identify columns with standard deviation of 0
zero_std_dev_columns = std_dev[std_dev == 0].index

print("Columns with standard deviation of 0 are being removed:")
for column in zero_std_dev_columns:
    print(column)
    
# Remove columns with standard deviation of 0
full_df = full_df.drop(columns=zero_std_dev_columns)

#Get rid of all rows with infs or nans
full_df = full_df.replace([np.inf, -np.inf], np.nan).dropna()

#get rid of negatives
for column in full_df.columns:
    full_df[column] = full_df[column].apply(lambda x: max(0, x) if pd.notnull(x) else x)
    
#Ensure all values are of type float
full_df = full_df.astype(float) 
full_df.head(10)

Columns with standard deviation of 0 are being removed:
 Active Std
 Idle Std


,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Active Mean,Active Max,Active Min,Idle Mean,Idle Max,Idle Min,label
0,10.0215,53913.0,216.582085,80.0,6.0,435.0,0.0,4597.701149,435.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.0215,53913.0,216.582085,80.0,6.0,259.0,0.0,7722.007722,259.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.0215,53913.0,216.582085,80.0,6.0,891.0,0.0,2244.668911,891.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.0215,53913.0,216.582085,80.0,6.0,1074.0,0.0,1862.197393,1074.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.0215,53913.0,216.582085,80.0,6.0,315.0,0.0,6349.206349,315.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,10.0215,53913.0,216.582085,80.0,6.0,4841.0,0.0,413.137782,4841.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,10.0215,53913.0,216.582085,80.0,6.0,581.0,0.0,3442.340792,581.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,10.0215,53913.0,216.582085,80.0,6.0,906.0,0.0,2207.505519,906.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,10.0215,53913.0,216.582085,80.0,6.0,401.0,0.0,4987.531172,401.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10.0215,53913.0,216.582085,80.0,6.0,760.0,0.0,2631.578947,760.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
features = full_df.columns.tolist()
features.remove('label')
print(features)

['Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Protocol', ' Flow Duration', ' Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Active Mean', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Max', ' Idle Min']


In [5]:
from scipy.stats import pointbiserialr
from math import sqrt

def getMerit(subset, label):
    k = len(subset)

    # average feature-class correlation
    rcf_all = []
    for feature in subset:
        if full_df[feature].std() == 0:
            print(feature)
        if full_df[feature].std() != 0:
            coeff = pointbiserialr( full_df[label], full_df[feature] )
            rcf_all.append( abs( coeff.correlation ) )
        rcf = np.mean( rcf_all )

    # average feature-feature correlation
    corr = full_df[subset].corr()
    corr.values[np.tril_indices_from(corr.values)] = np.nan
    corr = abs(corr)
    rff = corr.unstack().mean()

    return (k * rcf) / sqrt(k + k * (k-1) * rff)

In [6]:
class PriorityQueue:
    def __init__(self):
        self.queue = []

    def isEmpty(self):
        return len(self.queue) == 0
    
    def push(self, item, priority):
        """
        item already in priority queue with smaller priority:
        -> update its priority
        item already in priority queue with higher priority:
        -> do nothing
        if item not in priority queue:
        -> push it
        """
        for index, (i, p) in enumerate(self.queue):
            if set(i) == set(item):
                if p >= priority:
                    break
                del self.queue[index]
                self.queue.append((item, priority))
                self._heapify_up(len(self.queue) - 1)
                break
        else:
            self.queue.append((item, priority))
            self._heapify_up(len(self.queue) - 1)

    def pop(self):
        if not self.isEmpty():
            # swap the first and last elements
            self.queue[0], self.queue[-1] = self.queue[-1], self.queue[0]
            # pop the element with the highest priority
            item, priority = self.queue.pop()
            # restore the heap property
            self._heapify_down(0)
            return item, priority
        else:
            raise IndexError("pop from an empty priority queue")

    def _heapify_up(self, index):
        while index > 0:
            parent_index = (index - 1) // 2
            if self.queue[parent_index][1] < self.queue[index][1]:
                self.queue[parent_index], self.queue[index] = self.queue[index], self.queue[parent_index]
                index = parent_index
            else:
                break

    def _heapify_down(self, index):
        while True:
            left_child_index = 2 * index + 1
            right_child_index = 2 * index + 2
            largest = index

            if left_child_index < len(self.queue) and self.queue[left_child_index][1] > self.queue[largest][1]:
                largest = left_child_index

            if right_child_index < len(self.queue) and self.queue[right_child_index][1] > self.queue[largest][1]:
                largest = right_child_index

            if largest != index:
                self.queue[index], self.queue[largest] = self.queue[largest], self.queue[index]
                index = largest
            else:
                break


In [7]:
for _ in range(10):
    
    print("Iteration:", _ + 1)    
    best_value = -1
    best_feature = ''
    for feature in features:
        coeff = pointbiserialr(full_df[label], full_df[feature])
        abs_coeff = abs(coeff.correlation)

        if abs_coeff > best_value:
            best_value = abs_coeff
            best_feature = feature

    print("Best first feature is %s with merit %.4f"%(best_feature, best_value))

    # initialize queue
    queue = PriorityQueue()

    # push first tuple (subset, merit)
    queue.push([best_feature], best_value)

    # list for visited nodes
    visited = []

    # counter for backtracks
    n_backtrack = 0

    # limit of backtracks
    max_backtrack = 10

    # repeat until queue is empty
    # or the maximum number of backtracks is reached
    while not queue.isEmpty():
        # get element of queue with highest merit
        subset, priority = queue.pop()

        # check whether the priority of this subset
        # is higher than the current best subset
        if (priority < best_value):
            n_backtrack += 1
        else:
            best_value = priority
            best_subset = subset

        # goal condition
        if (n_backtrack == max_backtrack):
            break

        # iterate through all features and look of one can increase the merit
        for feature in features:
            temp_subset = subset + [feature]

            # check if this subset has already been evaluated
            for node in visited:
                if (set(node) == set(temp_subset)):
                    break
            # if not, ...
            else:
                # mark it as visited
                visited.append( temp_subset )
                # compute merit and push it to the queue
                merit = getMerit(temp_subset, label)
                queue.push(temp_subset, merit)
                
    features = [feature for feature in features if feature not in best_subset]
    print("Best Subset: ", best_subset)
    print("Best Value: ", best_value)
    print("\n")

Iteration: 1
Best first feature is  Protocol with merit 0.3700
Best Subset:  [' Protocol', ' Flow Duration']
Best Value:  0.44740985115009957


Iteration: 2
Best first feature is  Flow IAT Mean with merit 0.1313
Best Subset:  [' Flow IAT Mean', ' Source Port', ' Bwd IAT Max']
Best Value:  0.19307588007238985


Iteration: 3
Best first feature is  Flow IAT Min with merit 0.1283
Best Subset:  [' Flow IAT Min', ' Destination Port', ' Bwd IAT Std', ' Fwd IAT Min', ' Destination IP']
Best Value:  0.15522291104765906


Iteration: 4
Best first feature is Fwd IAT Mean with merit 0.0945
Best Subset:  ['Fwd IAT Mean', 'Fwd IAT Mean']
Best Value:  0.09448930007641097


Iteration: 5
Best first feature is Bwd IAT Mean with merit 0.0287
Best Subset:  ['Bwd IAT Mean', ' Flow Packets/s']
Best Value:  0.031665759696090076


Iteration: 6
Best first feature is  Fwd IAT Std with merit 0.0171
Best Subset:  [' Fwd IAT Std', ' Flow IAT Max', ' Flow Bytes/s', 'Source IP', ' Idle Min']
Best Value:  0.0178953549

In [8]:
print("Leftover features: ", features)

Leftover features:  []
